Welcome to your Hackathon !

Hackathon


Idea 1: Generative AI Content Pipeline with Workflow Automation


Objective:

Build an end-to-end automated pipeline that (1) generates text or images, (2) checks content quality, (3) applies ethical filtering, and (4) runs on modest CPU resources.


Step-by-Step Instructions

1. Define the Task & Pipeline Overview

    First, decide whether you’ll generate articles, summaries, or images.
    Then, sketch a high-level workflow:


Prompt → Generation Model → Quality-Check Module → (Optional) Image Generator → Ethical Filter → Output


2. Select Your Generation Method

    First, choose your text generation model family (e.g. Transformer-based) or image model (e.g. VAE/GAN).
    Then, justify why that family fits your goal (speed, quality, CPU constraints).


3. Pick Specific Pre-trained Models

    First, for text: pick a small model like distilGPT2 or t5-small.
    Then, for summarization: pick distilBERT or BART-base.
    Optionally, for images: choose a simple VAE (avoid heavy GANs on CPU).


4. Prepare & Subsample Your Dataset

    First, select a dataset (e.g. IMDB reviews for text, CIFAR-10 for images).
    Then, subsample to a manageable size (e.g. 5% of the full dataset).


5. Implement Text Generation Module

    First, write code to load your pre-trained LLM (e.g. via Hugging Face’s transformers).
    Then, create a function that takes a prompt and returns generated text.


6. Add Summarization for Quality Control

    First, load your BERT-based summarizer.
    Then, feed generated text into the summarizer and compare key points to the prompt to catch off-topic or low-quality outputs.


7. (Optional) Implement Image/Text Generation with VAE

    First, construct your VAE architecture for small images.
    Then, train or load a pre-trained VAE on your subsampled dataset.


8. Automate the Workflow

    First, choose an automation tool or scheduler (e.g. cron jobs, Airflow, or a simple Python script with schedule).
    Then, write a script that runs your generation + quality check + (optional) image step at fixed intervals or on demand.


9. Evaluate Your System

    First, generate a test set of 50–100 samples.
    Then, compute metrics:
        Text: BLEU, ROUGE, perplexity
        Robustness: adversarial prompts or noise injection


10. Integrate an Ethical Filter

    First, decide what to flag (e.g. bias, hate speech, misinformation).
    Then, plug in a simple classifier or rule-based detector after generation to filter or tag problematic outputs.


11. Document & Reflect

    First, write a one-page summary of your pipeline architecture and why you chose each component.
    Then, reflect on CPU trade-offs, quality outcomes, and ethical challenges.


What will you use?
Module 	Concepts / Libraries / Models
Generation 	Transformers; distilGPT2, t5-small; transformers library
Summarization QC 	BERT; distilBERT or BART-base; sentence-transformers
Image Module (optional) 	VAE; GAN (only on very small data); PyTorch or TensorFlow
Data Handling 	datasets library; subsampling techniques
Automation 	cron; Python schedule; Apache Airflow or similar
Evaluation Metrics 	BLEU; ROUGE; perplexity; adversarial testing
Ethical Filtering 	Bias detection; rule-based filters; simple classifier models
Optimization Tips 	small batch sizes; CPU-friendly model variants; distillation

PROMPT UTILISATEUR
    ↓
distilGPT2 (génération de texte)
    ↓
BART (résumé) → comparaison au prompt (cohérence)
    ↓
Filtre éthique (règles simples + classifieur si possible)
    ↓
Texte final + évaluation qualité

Famille choisie : Transformers
Critère	Justification
Vitesse	distilGPT2 et distilBART sont rapides sur CPU
Qualité	Bonne cohérence contextuelle et diversité des sorties
Contraintes CPU	Modèles distillés (moins de poids, chargement rapide)

Génération	distilgpt2	Léger, bon sur CPU, cohérent
Résumé / QC	sshleifer/distilbart-cnn-12-6	Résumé rapide et précis
Filtrage éthique	Détecteur naïf + classifieur	À construire avec regex + score (option SVM)
Tous les modèles seront chargés via Hugging Face Transformers.

Dataset : IMDB Movie Reviews

Texte libre, nombreux avis structurés

Utilisé comme base pour prompts ou pour tester le pipeline

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb", split="train[:5%]")  # ~1250 exemples
dataset = dataset.shuffle(seed=42).select(range(50))  # on en garde 50 pour test

c:\Users\mathi\Downloads\GenAI\GenAI_Bootcamp\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\mathi\Downloads\GenAI\GenAI_Bootcamp\nlp_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mathi\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In or

: 

On entre maintenant dans la phase d’exécution, à partir de l’Étape 5 : Text Generation que j’ai déjà partiellement codée plus haut, mais cette fois on la formalisera dans le rapport du hackathon avec tout le contexte, le code, les choix techniques et les tests.

Créer une fonction qui prend un prompt utilisateur, le passe à un modèle pré-entraîné (distilGPT2) et retourne un texte généré.
Choix technique :

    Modèle : distilgpt2

    Librairie : Hugging Face Transformers

    Périphérique : CPU uniquement

    Paramètres de génération : top_k, top_p, temperature pour plus de diversité

    Format : Fonction Python réutilisable dans un pipeline

Fonctionnement du module :

    Charger le tokenizer et le modèle distilGPT2

    Encodage du prompt en input IDs

    Génération avec sampling contrôlé

    Décodage en texte lisible

Pourquoi distilGPT2 ?

    Version distillée de GPT-2

    Compatible CPU

    Temps de génération faible (< 1s pour 80 tokens)

    Qualité suffisante pour des tests de workflow

Résumé automatique pour contrôle qualité
1. Rapport pédagogique – Étape 6
Objectif

Vérifier que le texte généré est pertinent par rapport au prompt d’origine.
On applique un résumé automatique au texte généré, puis on compare le résumé au prompt :

    Si le résumé ne contient aucun lien avec le prompt, le texte est probablement hors sujet.

    On pourra aussi calculer des similarités (étape suivante).

Choix du modèle

    Modèle : sshleifer/distilbart-cnn-12-6

    Pourquoi :

        Version distillée de BART (CPU-friendly)

        Entraîné sur tâche de résumé CNN/DailyMail

        Compatible Hugging Face Transformers

        Moins lourd que facebook/bart-base

Évaluation rapide
Critère	Résultat
Chargement modèle	~10 sec (CPU)
Qualité du résumé	Bonne condensation d’idées
Compatibilité CPU	Ok
Fiabilité comme QC	Moyenne seule — à coupler ensuite

Le résumé seul ne suffit pas à juger si le texte est correct → on ajoutera une comparaison par similarité dans l'étape suivante.

Ajouter une vérification automatique du type :

Compare(PROMPT, SUMMARY) → score de similarité → seuil

Vérification automatique de la qualité

Nous allons comparer le prompt initial et le résumé du texte généré à l’aide d’une similarité sémantique basée sur Sentence Transformers et cosine similarity.

Rapport pédagogique – Contrôle de cohérence
Objectif :

Évaluer automatiquement si le texte généré reste proche du prompt. On passe le résumé du texte généré et le prompt à un encodeur de phrase, puis on mesure leur similarité.
Outil utilisé :

    sentence-transformers/all-MiniLM-L6-v2

        Modèle léger pour embeddings de phrases

        Très rapide sur CPU

        Bon équilibre vitesse/précision

≥ 0.70	Bonne cohérence thématique
0.50 – 0.70	Cohérence moyenne / partielle
< 0.50	Résumé souvent hors-sujet ou trop vague

VAE (Variational Autoencoder) sur Images
Rapport pédagogique – Génération d’images par VAE
Objectif :

Compléter le pipeline par une génération d’images simples à partir de données visuelles comprimées, typiquement des chiffres manuscrits (MNIST).
Ce module est séparé du pipeline texte, mais montre la capacité à intégrer un autre générateur CPU-friendly.

Pourquoi VAE ?
Critère	Avantage
Léger et efficace	Fonctionne bien sur CPU
Interprétable	Apprentissage probabiliste + compression des images
Idéal sur MNIST	Format 28x28, rapide à entraîner/tester

Génération d’une image manuscrite aléatoire.

Exécutable même sur CPU sans CUDA.

Montre comment on peut élargir le pipeline IA à d’autres modalités.

RÉCAPITULATIF DU PIPELINE – Hackathon IA Générative CPU-Friendly
Objectif global

Créer un pipeline de génération de contenu automatique, avec contrôle qualité et filtrage éthique, 100% CPU compatible, en respectant une structure modulaire, claire et réutilisable.
Modules implémentés
Étape	Nom du module	Description
1	main.py	Orchestration du pipeline
2	data_loader.py	Chargement et sous-échantillonnage du dataset IMDB (5% puis 50 exemples)
3	generation.py	Génération de texte avec distilGPT2
4	summarization.py	Résumé automatique avec distilBART-cnn-12-6
5	similarity.py	Vérification de cohérence par similarité (prompt vs résumé)
6	vae_model.py	VAE simple pour images (MNIST, 28x28)
7	train_vae.py	Entraînement du VAE sur CPU
8	generate_image.py	Génération aléatoire d’image depuis l’espace latent

PROMPT UTILISATEUR
    ↓
distilGPT2 (génération)
    ↓
distilBART (résumé du texte généré)
    ↓
all-MiniLM (similarité résumé ↔ prompt)
    ↓
(score ∈ [0, 1]) → QC

Espace latent Z aléatoire
    ↓
Décodeur VAE (pré-entraîné)
    ↓
Image MNIST générée (28x28)

Génération de texte fonctionnelle

Résumé de qualité correcte

Score de similarité utilisable pour QC

VAE image testé avec succès

Tout est CPU-friendly, exécuté localement, bien commenté

Automate le Pipeline
Rapport pédagogique
Objectif :

Permettre une exécution complète du pipeline texte (de la génération à la vérification de cohérence), en un seul appel de script, pour :

    Tester rapidement plusieurs exemples

    Permettre une future exécution automatisée

    Garder des logs de résultats

python automate.py	Exécute immédiatement 3 exemples du pipeline texte
python automate_schedule.py	Planifie le pipeline toutes les X minutes (ajustable)

Évaluer le système
Rapport pédagogique
Objectif :

Évaluer la qualité du texte généré par le pipeline, et la robustesse du système face à des prompts tordus ou bruités.
Métriques utilisées
Type	Métrique	Utilité
Qualité texte	BLEU	Correspondance n-grammes entre référence/généré
	ROUGE-L	Similarité sur séquences longues (résumés)
Cohérence interne	Similarité cosinus	Déjà utilisée pour QC via embeddings
Robustesse	Adversarial prompts	Prompts perturbés pour tester stabilité
Mise en place d’un benchmark

On va :

    Générer 10 exemples (avec prompts IMDB)

    Pour chacun :

        Comparer prompt vs résumé (notre référence)

        Calculer BLEU et ROUGE-L

    Évaluer sur quelques prompts adversariaux

Filtrage éthique du contenu généré

Rapport pédagogique
Objectif :

Créer un filtre post-génération qui :

    Tague ou rejette les textes non conformes

    Se base sur des critères simples mais extensibles

    Fonctionne en local, sans API commerciale

Deux approches combinées :
Méthode	Avantage	Limite
Règles simples (regex/keywords)	Ultra rapide, contrôlable	Faille sur langage détourné
Classifieur léger (SVM ou NB)	Détection plus souple (ton, thème)	Entraînement minimal requis

# ✅ Étape 11 – Synthèse finale et réflexion

## 🎯 Objectifs atteints

| Objectif                                         | Statut     |
|--------------------------------------------------|------------|
| Génération de texte stable et rapide             | ✅ Réussi  |
| Résumé automatique pour contrôle                 | ✅ Réussi  |
| Comparaison résumé/prompt (similarité)           | ✅ Réussi  |
| Évaluation BLEU/ROUGE                            | ✅ Intégré |
| Génération image CPU avec VAE                    | ✅ Optionnel, fait |
| Filtrage éthique intégré                         | ✅ Par règles |
| Orchestration automatisée                        | ✅ CLI & Schedule |
| Modularité / testabilité                         | ✅ Bonne   |

---

## 📘 Apports pédagogiques

- Apprentissage de la **structure d’un pipeline NLP complet**
- Pratique du **multi-modèle Hugging Face**
- Maîtrise des **métriques d’évaluation** (BLEU, ROUGE)
- Introduction au **filtrage éthique local**
- Compréhension des **limites sur CPU**
- Réflexion sur la **cohérence générative automatique**

---

## ⚖️ Limites et pistes d'amélioration

| Limite                               | Suggestion d’amélioration               |
|--------------------------------------|-----------------------------------------|
| Filtre éthique basé sur mots-clés    | Ajouter un SVM ou BERT toxique local    |
| Prompt limité à 100 tokens           | Adapter dynamiquement selon modèle      |
| Aucun feedback utilisateur           | Ajouter une interface Streamlit         |
| Évaluation humaine absente           | Créer un tableau de scoring qualitatif  |